In [17]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## Create our QandA application

In [18]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [19]:
file = "../data/OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file, csv_args={"delimiter": ";"})
data = loader.load()

In [20]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders(
    [loader]
)

In [21]:
llm = ChatOpenAI(temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"},
)

### Coming up with test datapoints¶

In [22]:
data[1]

Document(page_content="name: CozyPrint Sweater Fleece Pullover\ndescription: The ultimate sweater fleece - made from superior fabric and offered at an unbeatable price.\\n\\nSize & Fit: Slightly Fitted: Softly shapes the body.\\n\\nWhy We Love It: Our customers (and employees) love the rugged construction and heritage-inspired styling of our popular Sweater Fleece Pullover and wear it for absolutely everything. From high-intensity activities to everyday tasks, you'll find yourself reaching for it every time.\\n\\nFabric & Care: Rugged sweater-knit exterior and soft brushed interior for exceptional warmth and comfort. Made in soft, 100% polyester. Machine wash and dry.\\n\\nAdditional Features: Features our classic Mount Katahdin logo. Front princess seams create a feminine shape. Kangaroo handwarmer pockets. Cuffs and hem reinforced with jersey binding. Imported.", metadata={'source': '../data/OutdoorClothingCatalog_1000.csv', 'row': 1})

In [23]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes",
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection",
    },
]

In [24]:
from langchain.evaluation.qa import QAGenerateChain

In [25]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [27]:
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in data[:5]])

[chain/start] [1:RunTypeEnum.chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [1:RunTypeEnum.chain:QAGenerateChain > 2:RunTypeEnum.llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content=\"name: Men's  Classic Denim, Standard Fit\\ndescription: Crafted from premium denim that will last wash after wash, our 1912 Standard Fit jeans are prewashed for unbeatable comfort right from the start. \\\\n\\\\nSize & Fit\\\\nStandard Fit: Sits lower on the waist and straight through the hip and thigh for a streamlined look.\\\\n\\\\nWhy We Love Them\\\\nCustomers rave 

ValueError: Could not parse output: QUESTION: What is the fabric composition of the CozyPrint Sweater Fleece Pullover?

ANSWER: The CozyPrint Sweater Fleece Pullover is made from 100% polyester.

In [28]:
new_examples[0]

{'query': "What is the fit and style of the Men's Classic Denim, Standard Fit jeans?",
 'answer': "The Men's Classic Denim, Standard Fit jeans have a Standard Fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look."}

In [29]:
examples += new_examples

In [30]:
import langchain
langchain.debug = True

In [31]:
qa.run(new_examples[0]["query"])

[chain/start] [1:RunTypeEnum.chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the fit and style of the Men's Classic Denim, Standard Fit jeans?"
}
[chain/start] [1:RunTypeEnum.chain:RetrievalQA > 2:RunTypeEnum.chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:RetrievalQA > 2:RunTypeEnum.chain:StuffDocumentsChain > 3:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the fit and style of the Men's Classic Denim, Standard Fit jeans?",
  "context": "name: Men's  Classic Denim, Standard Fit\ndescription: Crafted from premium denim that will last wash after wash, our 1912 Standard Fit jeans are prewashed for unbeatable comfort right from the start. \\n\\nSize & Fit\\nStandard Fit: Sits lower on the waist and straight through the hip and thigh for a streamlined look.\\n\\nWhy We Love Them\\nCustomers rave that these jeans \u201clook great and feel great\u201d while also telling us that

"The Men's Classic Denim, Standard Fit jeans have a standard fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look."

In [34]:
from langchain.evaluation.qa import QAEvalChain

In [38]:
predictions = qa.apply(examples)

[chain/start] [1:RunTypeEnum.chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "answer": "Yes"
}
[chain/start] [1:RunTypeEnum.chain:RetrievalQA > 2:RunTypeEnum.chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:RunTypeEnum.chain:RetrievalQA > 2:RunTypeEnum.chain:StuffDocumentsChain > 3:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "name: CozyPrint Sweater Fleece Pullover\ndescription: The ultimate sweater fleece - made from superior fabric and offered at an unbeatable price.\\n\\nSize & Fit: Slightly Fitted: Softly shapes the body.\\n\\nWhy We Love It: Our customers (and employees) love the rugged construction and heritage-inspired styling of our popular Sweater Fleece Pullover and wear it for absolutely everything. From high-intensity activities to everyday tasks, you'll fin

In [36]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [39]:
graded_outputs = eval_chain.evaluate(examples, predictions)

[chain/start] [1:RunTypeEnum.chain:QAEvalChain] Entering Chain run with input:
{
  "input_list": [
    {
      "query": "Do the Cozy Comfort Pullover Set        have side pockets?",
      "answer": "Yes",
      "result": "I'm sorry, but I don't have any information about the \"Cozy Comfort Pullover Set\" in my context."
    },
    {
      "query": "What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?",
      "answer": "The DownTek collection",
      "result": "I don't know."
    },
    {
      "query": "What is the fit and style of the Men's Classic Denim, Standard Fit jeans?",
      "answer": "The Men's Classic Denim, Standard Fit jeans have a Standard Fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look.",
      "result": "The Men's Classic Denim, Standard Fit jeans have a standard fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look."
    },


In [46]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print(f"Question: {predictions[i]['query']}")
    print(f"Real Answer: {predictions[i]['answer']}")
    print(f"Predicted answer: {predictions[i]['result']}")
    print(f"Predicted grade: {graded_outputs[i]['text']}")
    print("\n")

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted answer: I'm sorry, but I don't have any information about the "Cozy Comfort Pullover Set" in my context.
Predicted grade: INCORRECT


Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted answer: I don't know.
Predicted grade: INCORRECT


Example 2:
Question: What is the fit and style of the Men's Classic Denim, Standard Fit jeans?
Real Answer: The Men's Classic Denim, Standard Fit jeans have a Standard Fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look.
Predicted answer: The Men's Classic Denim, Standard Fit jeans have a standard fit, which means they sit lower on the waist and are straight through the hip and thigh for a streamlined look.
Predicted grade: CORRECT


Example 3:
Question: What is the name of the sweater fleece pull